In [ ]:
# Installing Roboflow and other dependencies¶
pip install -q --upgrade pip
pip install -q --upgrade diffusers transformers scipy ftfy huggingface_hub roboflow

In [ ]:
# Authenticating with the Hugging Face Hub

from huggingface_hub import notebook_login

# Required to get access to stable diffusion model
notebook_login()

In [ ]:
# Using the Hugging Face StableDiffusionPipeline Class

import torch
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16, revision="fp16"
)

pipeline = pipeline.to("cuda") #nividia A100

In [ ]:
# Creating Images Using Generate Images Function

import os

from IPython.display import Image, display

def generate_images(
    prompt,
    num_images_to_generate,
    num_images_per_prompt=4,
    guidance_scale=8,
    output_dir="generated_images",
    display_images=False,
):

    num_iterations = num_images_to_generate // num_images_per_prompt
    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_iterations):
        images = pipeline(
            prompt, num_images_per_prompt=num_images_per_prompt, guidance_scale=guidance_scale
        )
        for idx, image in enumerate(images.images):
            image_name = f"{output_dir}/image_{(i*num_images_per_prompt)+idx}.png"
            image.save(image_name)
            if display_images:
                display(Image(filename=image_name, width=128, height=128))

In [ ]:
# 1000 images takes 2-3 hours on a SageMaker Studio Lab GPU instance. 
# You can adjust the total image number below

generate_images("aerial view of cattle", 12, guidance_scale=4, display_images=True)

In [ ]:
# Here You Should ready with you Roboflow Account
# Push Generated Data to your Roboflow Project Directory using PIP Package
# Upload your Images to Roboflow via PIP package

In [ ]:
# Set Home path

import os
HOME = os.getcwd()

In [ ]:
from roboflow import Roboflow
import glob
import os

# glob params
image_dir = os.path.join(HOME, "generated_images", "")
file_extension_type = ".png"

# roboflow pip params
rf = Roboflow(api_key="YOUR_API_KEY")
upload_project = rf.workspace().project("YOUR_PROJECT_NAME")

# glob images
image_glob = glob.glob(image_dir + '/*' + file_extension_type)

# perform upload
for image in image_glob:
    upload_project.upload(image, num_retry_uploads=3)
    print("*** Processing image [" + str(len(image_glob)) + "] - " + image + " ***")

In [ ]:
# Check Images in RoboFlow Directory

In [ ]:
# Annotate the image for further modelling